In [1]:
# Cell 1: Install required packages
!pip install speechrecognition whisper-openai resampy soundfile librosa numba llvmlite scipy

  Using cached whisper_openai-1.0.0-py3-none-any.whl.metadata (480 bytes)
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
Using cached whisper_openai-1.0.0-py3-none-any.whl (1.2 MB)
Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
Using cached future-1.0.0-py3-none-any.whl (491 kB)

   ---------------------------------------- 0/3 [future]
   ---------------------------------------- 0/3 [future]
   ---------------------------------------- 0/3 [future]
   ---------------------------------------- 0/3 [future]
   ---------------------------------------- 0/3 [future]
   ---------------------------------------- 0/3 [future]
   ---------------------------------------- 0/3 [future]
   ---------------------------------------- 0/3 [future]
   ---------------------------------------- 0/3 [future]
   ---------------------------------------- 0/3 [future]
   ---------------------------------------- 0/3 [fut

In [79]:
# Cell 2: Import all required libraries
import speech_recognition as sr
import whisper
import numpy as np
import torch
import torchaudio
import librosa
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import json
import hashlib
from datetime import datetime
import warnings
import pickle
import os
from scipy import spatial
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

All libraries imported successfully!


In [80]:
# Cell 3: Initialize models and components with state persistence (FIXED BALANCE PERSISTENCE)
class BankingVoiceInterface:
    def __init__(self, state_file="banking_state.pkl"):
        self.state_file = state_file
        # Initialize speech recognition
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()
        
        # Load or initialize state
        self.load_state()
        
        # Initialize models (these will be loaded when needed)
        self.whisper_model = None
        self.voice_encoder = None
        self.intent_classifier = None
        
    def load_state(self):
        """Load state from file or initialize new state"""
        try:
            if os.path.exists(self.state_file):
                with open(self.state_file, 'rb') as f:
                    state = pickle.load(f)
                self.users = state['users']
                self.current_user = state['current_user']
                self.transactions = state['transactions']
                self.account_balance = state['account_balance']  # This should persist the balance
                print("✓ Previous session state loaded successfully!")
                print(f"✓ Current balance: INR {self.account_balance}")
                print(f"✓ Total transactions: {len(self.transactions)}")
            else:
                self.initialize_new_state()
                print("✓ New session initialized!")
        except Exception as e:
            print(f"Error loading state: {e}")
            self.initialize_new_state()
    
    def initialize_new_state(self):
        """Initialize fresh state"""
        self.users = {}
        self.current_user = None
        self.transactions = []
        self.account_balance = 10000  # INR - ONLY FOR FIRST TIME
        self.setup_sample_user()
    
    def save_state(self):
        """Save current state to file"""
        try:
            state = {
                'users': self.users,
                'current_user': self.current_user,
                'transactions': self.transactions,
                'account_balance': self.account_balance  # Make sure balance is saved
            }
            with open(self.state_file, 'wb') as f:
                pickle.dump(state, f)
            print("✓ Session state saved!")
        except Exception as e:
            print(f"Error saving state: {e}")
    
    def setup_sample_user(self):
        # Create a sample user with voice enrollment
        sample_user = {
            'user_id': 'user123',
            'voice_embedding': None,  # Will be set during enrollment
            'pin': self.hash_pin('1234'),
            'balance': 10000,  # This is separate from account_balance
            'transactions': [
                {'date': '2024-01-15', 'type': 'debit', 'amount': 1500, 'description': 'Shopping'},
                {'date': '2024-01-10', 'type': 'debit', 'amount': 2000, 'description': 'Dining'},
                {'date': '2024-01-05', 'type': 'credit', 'amount': 5000, 'description': 'Salary'},
                {'date': '2024-01-01', 'type': 'debit', 'amount': 1000, 'description': 'Groceries'},
                {'date': '2023-12-28', 'type': 'debit', 'amount': 3000, 'description': 'Travel'}
            ]
        }
        self.users['user123'] = sample_user

print("Banking interface class defined with proper balance persistence!")

Banking interface class defined with state persistence!


In [81]:
# Cell 4: Core functionality methods (FIXED WHISPER FILE ISSUE)
class BankingVoiceInterface(BankingVoiceInterface):
    def hash_pin(self, pin):
        """Hash PIN for security"""
        return hashlib.sha256(pin.encode()).hexdigest()
    
    def record_audio(self, duration=5, filename="temp_audio.wav"):
        """Record audio from microphone with proper file handling"""
        print(f"Recording for {duration} seconds...")
        try:
            with self.microphone as source:
                self.recognizer.adjust_for_ambient_noise(source)
                print("Start speaking now...")
                audio = self.recognizer.record(source, duration=duration)
            
            # Save audio to file with full path
            import os
            filepath = os.path.abspath(filename)
            with open(filepath, "wb") as f:
                f.write(audio.get_wav_data())
            
            print(f"✓ Recording saved to: {filepath}")
            return filepath
        except Exception as e:
            print(f"Recording error: {e}")
            return None
    
    def speech_to_text(self, audio_filepath):
        """Convert speech to text - FIXED WHISPER ISSUE"""
        try:
            # Try Google speech recognition first (more reliable)
            print("Using Google speech recognition...")
            with sr.AudioFile(audio_filepath) as source:
                audio = self.recognizer.record(source)
                text = self.recognizer.recognize_google(audio)
                print(f"Recognized text: '{text}'")
                return text
        except Exception as e:
            print(f"Google speech recognition error: {e}")
            # Fallback to Whisper
            try:
                print("Trying Whisper as fallback...")
                if self.whisper_model is None:
                    self.whisper_model = whisper.load_model("base")
                
                result = self.whisper_model.transcribe(audio_filepath)
                text = result["text"].strip()
                print(f"Whisper fallback text: '{text}'")
                return text
            except Exception as e2:
                print(f"Whisper fallback also failed: {e2}")
                return None
    
    def create_voice_embedding(self, audio_path):
        """Create voice embedding for biometric authentication"""
        try:
            import os
            if not os.path.exists(audio_path):
                print(f"Audio file not found for embedding: {audio_path}")
                return None
                
            wav, sr = librosa.load(audio_path, sr=16000)
            mfcc = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=13)
            embedding = np.mean(mfcc, axis=1)
            print("✓ Voice embedding created successfully")
            return embedding
        except Exception as e:
            print(f"Voice embedding error: {e}")
            return None
    
    def verify_voiceprint(self, current_embedding, stored_embedding, threshold=0.7):
        """Verify voiceprint similarity"""
        if stored_embedding is None:
            return True  # First time enrollment
        
        try:
            similarity = 1 - spatial.distance.cosine(current_embedding, stored_embedding)
            print(f"Voice similarity score: {similarity:.4f}")
            return similarity > threshold
        except Exception as e:
            print(f"Voice verification error: {e}")
            return False
    
    def classify_intent(self, text):
        """Classify user intent from text"""
        try:
            text_lower = text.lower()
            
            if any(word in text_lower for word in ['transfer', 'send', 'pay']):
                return 'transfer'
            elif any(word in text_lower for word in ['balance', 'check balance']):
                return 'balance'
            elif any(word in text_lower for word in ['transaction', 'history', 'statement']):
                return 'transactions'
            elif any(word in text_lower for word in ['pin', 'authenticate', 'change pin']):
                return 'change_pin'
            else:
                return 'unknown'
        except:
            return 'unknown'

print("Core methods with fixed speech recognition added!")

Core methods with fixed speech recognition added!


In [82]:
# Cell 5: Banking operations (FIXED BALANCE SYNCHRONIZATION)
class BankingVoiceInterface(BankingVoiceInterface):
    def process_transfer(self, command_text):
        """Process transfer command"""
        try:
            # Extract amount and recipient from command
            words = command_text.split()
            amount = None
            recipient = None
            
            for i, word in enumerate(words):
                if word.startswith('$') or word.replace('.', '').isdigit():
                    amount = float(word.replace('$', ''))
                if word.lower() in ['to', 'for'] and i + 1 < len(words):
                    recipient = ' '.join(words[i+1:])
                    break
            
            if amount is None or recipient is None:
                return "Could not understand transfer details. Please specify amount and recipient."
            
            if amount > self.account_balance:
                return f"Insufficient funds. Available balance: INR {self.account_balance}"
            
            # Process transfer - UPDATE BOTH account_balance AND user balance
            self.account_balance -= amount
            self.users['user123']['balance'] = self.account_balance  # Sync user balance
            
            transaction = {
                'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'type': 'debit',
                'amount': amount,
                'description': f'Transfer to {recipient}',
                'balance_after': self.account_balance
            }
            self.transactions.append(transaction)
            self.save_state()  # Save state after transaction
            
            return f"Successfully transferred INR {amount} to {recipient}. New balance: INR {self.account_balance}"
            
        except Exception as e:
            return f"Error processing transfer: {str(e)}"
    
    def check_balance(self):
        """Check account balance"""
        # Ensure both balances are synced
        if self.account_balance != self.users['user123']['balance']:
            self.users['user123']['balance'] = self.account_balance
            self.save_state()
            
        return f"Your current account balance is: INR {self.account_balance}"
    
    def show_transactions(self, count=5):
        """Show recent transactions"""
        all_transactions = self.users['user123']['transactions'] + self.transactions
        recent_txns = all_transactions[-count:] if all_transactions else []
        
        if not recent_txns:
            return "No recent transactions found."
        
        result = f"Last {len(recent_txns)} transactions:\n"
        for txn in recent_txns:
            result += f"{txn['date']} - {txn['type'].upper()} - INR {txn['amount']} - {txn['description']}\n"
        
        return result

print("Banking operations with balance synchronization added!")

Banking operations added!


In [83]:
# Cell 6: Security and authentication layer (WITH PIN CHANGE FEATURE)
class BankingVoiceInterface(BankingVoiceInterface):
    def enroll_user_voice(self):
        """Enroll user's voice for biometric authentication"""
        print("Please speak the following phrase for voice enrollment: 'My voice is my password'")
        input("Press Enter when ready...")
        
        audio_file = self.record_audio(duration=4, filename="enrollment_audio.wav")
        if not audio_file:
            print("❌ Failed to record enrollment audio")
            return False
        
        embedding = self.create_voice_embedding(audio_file)
        if embedding is not None:
            self.users['user123']['voice_embedding'] = embedding.tolist()
            self.save_state()
            print("✓ Voice enrollment completed successfully!")
            return True
        else:
            print("❌ Failed to create voice embedding")
            return False
    
    def change_pin_voice(self):
        """Change PIN using voice commands"""
        print("\n=== CHANGE PIN ===")
        print("First, verify your current PIN...")
        
        # Verify current PIN
        current_pin = input("Enter your current 4-digit PIN: ").strip()
        if self.hash_pin(current_pin) != self.users['user123']['pin']:
            print("❌ Current PIN incorrect")
            return False
        
        print("✓ Current PIN verified!")
        print("\nNow set your new PIN using voice...")
        print("Speak your new 4-digit PIN clearly (e.g., 'five six seven eight')")
        input("Press Enter when ready to speak new PIN...")
        
        max_attempts = 3
        for attempt in range(max_attempts):
            new_pin_audio = self.record_audio(duration=4, filename=f"new_pin_{attempt}.wav")
            if not new_pin_audio:
                print("❌ Failed to record new PIN audio")
                continue
                
            new_pin_text = self.speech_to_text(new_pin_audio)
            
            if new_pin_text:
                print(f"Recognized: '{new_pin_text}'")
                
                # Convert spoken numbers to digits
                number_words = {
                    'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
                    'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9'
                }
                
                new_pin_digits = ""
                words = new_pin_text.lower().split()
                
                for word in words:
                    clean_word = word.strip('.,!?')
                    if clean_word in number_words:
                        new_pin_digits += number_words[clean_word]
                    elif clean_word.isdigit():
                        new_pin_digits += clean_word
                
                if len(new_pin_digits) == 4:
                    print(f"✓ New PIN recognized: {new_pin_digits}")
                    
                    # Confirm new PIN
                    print("\nPlease confirm your new PIN by speaking it again...")
                    input("Press Enter to confirm...")
                    
                    confirm_audio = self.record_audio(duration=4, filename=f"confirm_pin_{attempt}.wav")
                    if confirm_audio:
                        confirm_text = self.speech_to_text(confirm_audio)
                        if confirm_text:
                            confirm_digits = ""
                            confirm_words = confirm_text.lower().split()
                            for word in confirm_words:
                                clean_word = word.strip('.,!?')
                                if clean_word in number_words:
                                    confirm_digits += number_words[clean_word]
                                elif clean_word.isdigit():
                                    confirm_digits += clean_word
                            
                            if confirm_digits == new_pin_digits:
                                # Update PIN
                                self.users['user123']['pin'] = self.hash_pin(new_pin_digits)
                                self.save_state()
                                print(f"✓ PIN successfully changed to: {new_pin_digits}")
                                return True
                            else:
                                print("❌ PIN confirmation failed. PINs don't match.")
                        else:
                            print("❌ Could not recognize confirmation PIN")
                    else:
                        print("❌ Failed to record confirmation PIN")
                    
                    break
                else:
                    print(f"Could not recognize 4-digit PIN. Got: '{new_pin_digits}'")
                    if attempt < max_attempts - 1:
                        print("Please try speaking your new PIN again...")
                    else:
                        print("Switching to manual PIN entry...")
                        manual_pin = input("Please type your new 4-digit PIN: ").strip()
                        confirm_pin = input("Confirm new PIN: ").strip()
                        if manual_pin == confirm_pin and len(manual_pin) == 4:
                            self.users['user123']['pin'] = self.hash_pin(manual_pin)
                            self.save_state()
                            print(f"✓ PIN successfully changed to: {manual_pin}")
                            return True
                        else:
                            print("❌ PIN change failed")
                            return False
            else:
                print("No PIN detected.")
                if attempt < max_attempts - 1:
                    print("Please try speaking your new PIN again...")
        
        return False
    
    def authenticate_user(self):
        """Two-factor authentication: Voiceprint + PIN"""
        print("Voice authentication required...")
        print("Please speak: 'My voice is my password'")
        
        # First factor: Voice biometrics
        auth_audio_file = self.record_audio(duration=4, filename="auth_audio.wav")
        if not auth_audio_file:
            return False, "Failed to record authentication audio"
        
        current_embedding = self.create_voice_embedding(auth_audio_file)
        if current_embedding is None:
            return False, "Failed to create voice embedding"
            
        stored_embedding = self.users['user123']['voice_embedding']
        
        if stored_embedding is not None:
            stored_embedding = np.array(stored_embedding)
        
        voice_match = self.verify_voiceprint(current_embedding, stored_embedding)
        
        if not voice_match:
            return False, "Voice authentication failed"
        
        # Second factor: PIN verification
        print("✓ Voice authentication successful! Now speak your 4-digit PIN...")
        print("Speak clearly, for example: 'one two three four' or '1 2 3 4'")
        input("Press Enter when ready to speak PIN...")
        
        max_attempts = 2
        pin_text = None
        
        for attempt in range(max_attempts):
            pin_audio_file = self.record_audio(duration=4, filename=f"pin_audio_{attempt}.wav")
            if not pin_audio_file:
                print("❌ Failed to record PIN audio")
                continue
                
            pin_text = self.speech_to_text(pin_audio_file)
            
            if pin_text:
                print(f"Recognized: '{pin_text}'")
                
                number_words = {
                    'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
                    'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9'
                }
                
                pin_digits = ""
                words = pin_text.lower().split()
                
                for word in words:
                    clean_word = word.strip('.,!?')
                    if clean_word in number_words:
                        pin_digits += number_words[clean_word]
                    elif clean_word.isdigit():
                        pin_digits += clean_word
                
                if len(pin_digits) == 4:
                    pin_text = pin_digits
                    print(f"✓ Extracted PIN: {pin_text}")
                    break
                else:
                    print(f"Could not recognize 4-digit PIN clearly. Got: '{pin_digits}'")
                    if attempt < max_attempts - 1:
                        print("Please try speaking your PIN again...")
                    else:
                        print("Switching to manual PIN entry...")
                        pin_text = input("Please type your 4-digit PIN: ").strip()
            else:
                print("No PIN detected in audio.")
                if attempt < max_attempts - 1:
                    print("Please try speaking your PIN again...")
                else:
                    pin_text = input("Please type your 4-digit PIN: ").strip()
        
        # Verify PIN
        if pin_text and self.hash_pin(pin_text.strip()) == self.users['user123']['pin']:
            self.current_user = 'user123'
            self.save_state()
            return True, "Authentication successful!"
        else:
            return False, "PIN verification failed"

print("Security layer with PIN change feature added!")

Security layer with PIN change feature added!


In [84]:
# Cell 7: Main interface and command processor (SHOW COMMANDS BEFORE EVERY INPUT)
class BankingVoiceInterface(BankingVoiceInterface):
    def process_command(self, command_text):
        """Process voice command and execute appropriate action"""
        if not command_text:
            return "No command detected. Please try again."
            
        intent = self.classify_intent(command_text)
        print(f"Detected intent: {intent}")
        
        if intent == 'transfer':
            return self.process_transfer(command_text)
        elif intent == 'balance':
            return self.check_balance()
        elif intent == 'transactions':
            return self.show_transactions()
        elif intent == 'change_pin':
            success = self.change_pin_voice()
            return "PIN changed successfully!" if success else "PIN change failed"
        else:
            return "Sorry, I didn't understand that command. Please try: 'Transfer amount to person', 'Check balance', 'Show transactions', or 'Change PIN'"
    
    def show_available_commands(self):
        """Display available commands"""
        print("\n" + "="*50)
        print("🎯 AVAILABLE VOICE COMMANDS:")
        print("="*50)
        print("💰 Banking Commands:")
        print("  - 'Transfer [amount] to [recipient]'")
        print("  - 'Check my balance'")
        print("  - 'Show my transactions'")
        print("  - 'Show last 5 transactions'")
        print("🔐 Security Commands:")
        print("  - 'Change PIN'")
        print("  - 'Exit' to quit")
        print("="*50)
    
    def run_interface(self):
        """Main interface loop"""
        print("=== Secure Voice Banking Interface ===")
        print("Initializing system...")
        
        # Initialize models
        if self.whisper_model is None:
            print("Loading speech recognition...")
            print("✓ Speech recognition ready")
        
        # First-time voice enrollment
        if self.users['user123']['voice_embedding'] is None:
            print("\nFirst-time setup: Voice enrollment required")
            enrollment_success = self.enroll_user_voice()
            if not enrollment_success:
                print("❌ Voice enrollment failed. Please restart.")
                return
        
        print("\nSystem ready! Starting authentication...")
        
        # Two-factor authentication
        auth_success, auth_message = self.authenticate_user()
        
        if not auth_success:
            print(f"❌ Authentication failed: {auth_message}")
            return
        
        print(f"\n✓ {auth_message}")
        
        # Show commands at the start
        self.show_available_commands()
        
        # Main command loop
        command_count = 0
        while True:
            print(f"\n--- Command #{command_count + 1} ---")
            print("🎤 Speak your command now...")
            input("Press Enter to start recording...")
            
            try:
                audio_file = self.record_audio(duration=6, filename=f"command_{command_count}.wav")
                if not audio_file:
                    print("❌ Recording failed. Please try again.")
                    continue
                
                command_text = self.speech_to_text(audio_file)
                command_count += 1
                
                if command_text:
                    print(f"👤 You said: '{command_text}'")
                    
                    if 'exit' in command_text.lower() or 'quit' in command_text.lower():
                        print("Thank you for using Voice Banking!")
                        self.save_state()
                        break
                    
                    response = self.process_command(command_text)
                    print(f"🤖 Response: {response}")
                    
                    # Show commands again after each command
                    self.show_available_commands()
                    
                else:
                    print("❌ No speech detected. Please try again.")
                    # Show commands again if no speech detected
                    self.show_available_commands()
                    
            except Exception as e:
                print(f"❌ Error: {e}")
                # Show commands again after error
                self.show_available_commands()

print("Main interface with always-visible commands ready!")

Main interface with PIN change option ready!


In [85]:
# Cell 8: Initialize and test the system (UPDATED WITH MIC CHECK)
# Create banking interface instance - this will load previous state automatically
banking_app = BankingVoiceInterface()

print("Banking Voice Interface initialized successfully!")
print(f"Initial account balance: INR {banking_app.account_balance}")
print(f"Voice enrolled: {banking_app.users['user123']['voice_embedding'] is not None}")
print(f"Total transactions: {len(banking_app.transactions)}")
print(f"Default PIN: 1234")

# Test microphone availability
print("\n🔍 Checking microphone...")
try:
    with banking_app.microphone as source:
        banking_app.recognizer.adjust_for_ambient_noise(source, duration=1)
    print("✓ Microphone is available and working")
except Exception as e:
    print(f"❌ Microphone issue: {e}")
    print("Please check your microphone permissions and connections")

✓ Previous session state loaded successfully!
Banking Voice Interface initialized successfully!
Initial account balance: INR 5000.0
Voice enrolled: True
Total transactions: 1
Default PIN: 1234

🔍 Checking microphone...
✓ Microphone is available and working


In [86]:
# Cell 9: Test individual components (SAME AS BEFORE)
def test_speech_recognition():
    """Test speech recognition component"""
    print("Testing speech recognition...")
    print("Please speak a test phrase after pressing Enter...")
    input("Press Enter to start recording...")
    
    audio = banking_app.record_audio(duration=4)
    text = banking_app.speech_to_text(audio)
    
    print(f"Recognized text: {text}")
    return text

# Uncomment to test
# test_speech_recognition()

In [87]:
# Cell 10: Test voice enrollment (SAME AS BEFORE)
def test_voice_enrollment():
    """Test voice enrollment process"""
    print("Testing voice enrollment...")
    banking_app.enroll_user_voice()
    print("Voice enrollment completed!")

# Uncomment to test
# test_voice_enrollment()

In [88]:
# Cell 11: Test authentication (UPDATED FOR NEW FLOW)
def test_authentication():
    """Test two-factor authentication"""
    print("Testing authentication...")
    print("Note: This will test the complete authentication flow")
    print("You'll need to:")
    print("1. Speak 'My voice is my password' for voice auth")
    print("2. Speak your PIN (try saying 'one two three four' or '1234')")
    print("3. If voice PIN fails, you can type '1234'")
    input("Press Enter to start authentication test...")
    
    success, message = banking_app.authenticate_user()
    print(f"Authentication result: {success} - {message}")

# Uncomment to test
# test_authentication()

In [89]:
# Cell 12: Run the complete banking interface (UPDATED INSTRUCTIONS)
print("Starting Secure Voice Banking Interface...")
print("NOTE: Authentication Process:")
print("1. Voice Recognition: Speak 'My voice is my password'")
print("2. PIN Entry: Speak your 4-digit PIN (try 'one two three four' or '1234')")
print("3. Fallback: If voice PIN fails, you can type '1234'")
print("\nDefault PIN is: 1234")
input("Press Enter to start...")

# Run the main interface
banking_app.run_interface()

Starting Secure Voice Banking Interface...
NOTE: Authentication Process:
1. Voice Recognition: Speak 'My voice is my password'
2. PIN Entry: Speak your 4-digit PIN (try 'one two three four' or '1234')
3. Fallback: If voice PIN fails, you can type '1234'

Default PIN is: 1234


Press Enter to start... 1234


=== Secure Voice Banking Interface ===
Initializing system...
Loading speech recognition...
✓ Speech recognition ready

System ready! Starting authentication...
Voice authentication required...
Please speak: 'My voice is my password'
Recording for 4 seconds...
Start speaking now...
✓ Recording saved to: C:\Users\Girijesh\auth_audio.wav
✓ Voice embedding created successfully
Voice similarity score: 0.9968
✓ Voice authentication successful! Now speak your 4-digit PIN...
Speak clearly, for example: 'one two three four' or '1 2 3 4'


Press Enter when ready to speak PIN... 1234


Recording for 4 seconds...
Start speaking now...
✓ Recording saved to: C:\Users\Girijesh\pin_audio_0.wav
Using Google speech recognition...
Recognized text: '1234'
Recognized: '1234'
✓ Extracted PIN: 1234
✓ Session state saved!

✓ Authentication successful!

Available commands:
- 'Transfer [amount] to [recipient]'
- 'Check balance'
- 'Show transactions'
- 'Change PIN' to change your PIN
- 'Exit' to quit

--- Command #1 ---
Speak your command now...


...

Recording for 4 seconds...
Start speaking now...
✓ Recording saved to: C:\Users\Girijesh\confirm_pin_0.wav
Using Google speech recognition...
Recognized text: '4567'
✓ Session state saved!
✓ PIN successfully changed to: 4567
🤖 Response: PIN changed successfully!

--- Command #3 ---
Speak your command now...


Press Enter to start recording... 


Recording for 6 seconds...
Start speaking now...
✓ Recording saved to: C:\Users\Girijesh\command_2.wav
Using Google speech recognition...
Recognized text: 'check balance'
👤 You said: 'check balance'
Detected intent: balance
🤖 Response: Your current account balance is: INR 5000.0

--- Command #4 ---
Speak your command now...


Press Enter to start recording... 


Recording for 6 seconds...
Start speaking now...
✓ Recording saved to: C:\Users\Girijesh\command_3.wav
Using Google speech recognition...
Recognized text: 'exit'
👤 You said: 'exit'
Thank you for using Voice Banking!
✓ Session state saved!
